In [1]:
from pathlib import Path
import glob
import numpy as np
import tables 
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from scipy.ndimage.measurements import center_of_mass
from tqdm.notebook import tqdm

In [2]:
RESULTS_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn'
WORKING_DIR = os.getcwd()
CODE_DIR = os.path.abspath(os.path.join(WORKING_DIR, os.pardir))
h5py_path_list = [h5py for h5py in glob.glob(RESULTS_DIR + '/*h5')]
h5py_path_list.sort()
h5py_path_list

['/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/beach_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/breite_strasse_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/bridge_1_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/bridge_2_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/bumblebee_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/doves_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/ducks_boat_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/ducks_children_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/golf_out.h5',
 '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/mrcnn/holsten_gate_out.h5',
 '/Users/timschroder/

In [4]:
DATASET = 'LUEBECK/'
hpc = False

DIR = Path(os.path.dirname(os.path.realpath('__file__')))#.parent
if (DIR== Path('/beegfs/home/users/t/tim.schroeder/')):
    CODE_DIR = os.path.join(DIR, 'object_rep/project_code/') 
    hpc = True
else: 
    CODE_DIR = DIR

print('CODE_DIR: ', CODE_DIR)
PROJECT_DIR = Path(CODE_DIR).parent
print('PROJECT_DIR: ', PROJECT_DIR)
DATA_DIR = os.path.join(PROJECT_DIR, 'project_data.nosync/', DATASET)
print('DATA_DIR: ', DATA_DIR)
RESULTS_DIR = os.path.join(PROJECT_DIR, 'project_results.nosync/',DATASET)
D_DIR = os.path.join(PROJECT_DIR, 'project_results.nosync/',DATASET,'panoptic-seg/')
print('RESULTS_DIR: ', RESULTS_DIR)

DEEPSORT_DIR = os.path.join(CODE_DIR, "deep_sort/")
print('DEEPSORT_DIR: ', DEEPSORT_DIR)

sys.path.append(DEEPSORT_DIR)

video_path_list = [video for video in glob.glob(DATA_DIR + '/*')]
video_path_list.sort()

h5py_path_list = [video for video in glob.glob(RESULTS_DIR + 'mrcnn/*.h5')]
h5py_path_list.sort()
h5py_path_list

CODE_DIR:  /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/final_code
PROJECT_DIR:  /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code
DATA_DIR:  /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/project_data.nosync/LUEBECK/
RESULTS_DIR:  /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/project_results.nosync/LUEBECK/
DEEPSORT_DIR:  /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/final_code/deep_sort/


[]

## Save single frames from videos for DEEP Sort

In [86]:
import cv2
for video_path in video_path_list:
    
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    print(video_name)
    VID_DIR = DEEPSORT_DIR+'LUEBECK.nosync/'+ video_name
    if os.path.exists(VID_DIR) == False:
        os.mkdir(VID_DIR)   
    
    vidcap = cv2.VideoCapture(video_path)
    success,image = vidcap.read()
    
    IMG_DIR = VID_DIR + '/img1/'
    if os.path.exists(IMG_DIR) == False:
        os.mkdir(IMG_DIR) 
    count = 0
    while success:
        cv2.imwrite(IMG_DIR+'%06d.jpg' % count, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        count += 1
    
    vidcap.release(); del vidcap

st_petri_mcdonalds
street


## Convert .h5py (from MRCNN) --> .txt (for DEEP SORT)

In [5]:
for filename in h5py_path_list:
    video_name = os.path.splitext(os.path.basename(filename))[0]
    video_name = video_name[:-4]
    print(video_name)
    VID_DIR = DEEPSORT_DIR+'LUEBECK.nosync/'+ video_name
    OUT_DIR = VID_DIR + '/det/'
    if os.path.exists(OUT_DIR) == False:
        os.mkdir(OUT_DIR) 

    f = tables.open_file(filename, mode='r')
    pred_boxes = f.root.pred_boxes[:]
    scores = f.root.scores[:]
    f.close()

    frame_id = range(0,int(scores.shape[0]/100))
    frame_nr = np.repeat(frame_id,100).reshape(len(frame_id)*100,1)
    
    for i in range(len(pred_boxes)):
        box = pred_boxes[i]
        x_min = min(box[0],box[2])
        x_max = max(box[0],box[2])
        y_min = min(box[1],box[3])
        y_max = max(box[1],box[3])
        w = x_max - x_min
        h = y_max - y_min
        pred_boxes[i] = [x_min,y_min,w,h]

    results = np.concatenate((frame_nr,pred_boxes, scores), axis=1)
    results = results[~np.isnan(results).any(axis=1)]
    output_file = OUT_DIR + 'det.txt'
    print(output_file)
    f = open(output_file, 'w')
    for row in results:
        print('%d,-1,%.2f,%.2f,%.2f,%.2f,%.2f,-1,-1,-1' % (row[0], row[1], row[2], row[3], row[4], row[5]),file=f)
  

IndexError: list index out of range

## Generate DEEP Sort Detections (.npy) from det.txt

In [97]:
!python /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/tools/generate_detections.py --model=/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/resources/networks/mars-small128.pb --mot_dir=/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/Test --output_dir=/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/Test/

5
Frame 00082/00595
Frame 00083/00595
Frame 00084/00595
Frame 00085/00595
Frame 00086/00595
Frame 00087/00595
Frame 00088/00595
Frame 00089/00595
Frame 00090/00595
Frame 00091/00595
Frame 00092/00595
Frame 00093/00595
Frame 00094/00595
Frame 00095/00595
Frame 00096/00595
Frame 00097/00595
Frame 00098/00595
Frame 00099/00595
Frame 00100/00595
Frame 00101/00595
Frame 00102/00595
Frame 00103/00595
Frame 00104/00595
Frame 00105/00595
Frame 00106/00595
Frame 00107/00595
Frame 00108/00595
Frame 00109/00595
Frame 00110/00595
Frame 00111/00595
Frame 00112/00595
Frame 00113/00595
Frame 00114/00595
Frame 00115/00595
Frame 00116/00595
Frame 00117/00595
Frame 00118/00595
Frame 00119/00595
Frame 00120/00595
Frame 00121/00595
Frame 00122/00595
Frame 00123/00595
Frame 00124/00595
Frame 00125/00595
Frame 00126/00595
Frame 00127/00595
Frame 00128/00595
Frame 00129/00595
Frame 00130/00595
Frame 00131/00595
Frame 00132/00595
Frame 00133/00595
Frame 00134/00595
Frame 00135/00595
Frame 00136/00595
Frame 00

## Run deep_sort_app.py on all videos

In [2]:
!python /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/deep_sort_app.py     --sequence_dir=/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/LUEBECK.nosync/sea     --detection_file=/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/resources/detections/LUEBECK_mrcnn/sea.npy --output_file=/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/detections/sea.txt --min_confidence=0.7    --nn_budget=200 --max_cosine_distance=0.9   --nms_max_overlap=1 	--display=False 

ackages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
Processing frame 00536
/Applications/anaconda3/envs/torchenv/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/Applications/anaconda3/envs/torchenv/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
Processing frame 00537
/Applications/anaconda3/envs/torchenv/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is de

## Combine Deepsort bounding boxes with panoptic segmentation results

In [7]:
# Load Deepsort bboxes
DETECT_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/detections'
detection_path_list = [video for video in glob.glob(DETECT_DIR + '/*.txt')]
detection_path_list.sort()


In [8]:
D_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/panoptic-seg/'
data_path_list = [video for video in glob.glob(D_DIR + '*_data.npy')]
data_path_list.sort()

info_path_list = [video for video in glob.glob(D_DIR + '*_info.npy')]
info_path_list.sort()


In [9]:
len(data_path_list)

18

### Updating segmentation_info with track_id

In [10]:
def box_to_tlbr(bbox):
    bbox[2:] = bbox[:2] + bbox[2:]
    return bbox

In [11]:
def box_to_mask(bbox):
    x_max = bbox[2]
    x_min = bbox[0]
    y_max = bbox[3]
    y_min = bbox[1]
    box_mask = np.fromfunction(lambda y, x: (x <= x_max) * (x >= x_min) * (y <= y_max) * (y >= y_min), (720,1280), dtype=int)
    return box_mask

In [12]:
def center_distance(box,pan_mask):
    bbox = box_to_tlbr(box.copy())
    box_mask = box_to_mask(bbox)    
    center_pan_mask = center_of_mass(pan_mask)
    center_box_mask = center_of_mass(box_mask)
    dist = euclidean(center_pan_mask,center_box_mask)
    return dist

In [13]:

for v in tqdm([17]):
    print(data_path_list[v])
    print(info_path_list[v])
    print(detection_path_list[v])
    panoptic_seg = np.load(data_path_list[v], allow_pickle=True) # change storage.py line 134 for this
    segments_info = np.load(info_path_list[v],allow_pickle=True)
    detection_file = detection_path_list[v]
    detections_in = np.loadtxt(detection_file, delimiter=',')
    if (len(detections_in) != 0):
        frame_indices = detections_in[:, 0].astype(np.int)
        min_frame_idx = frame_indices.astype(np.int).min()
        max_frame_idx = frame_indices.astype(np.int).max()
        for frame_idx in tqdm(range(min_frame_idx, max_frame_idx + 1)):
            #print("Frame %05d/%05d" % (frame_idx, max_frame_idx))
            mask = frame_indices == frame_idx
            rows = detections_in[mask]
            pan_mask = panoptic_seg[frame_idx].numpy()
            pan_ids = np.unique(pan_mask)[1:] #cut 0 in beginning
            for row in rows:
                track_id = row[1].astype(int)
                box = row[2:6].astype(int)
                distances = []#np.array(2,np.unique(pan_mask))
                for pan_id in pan_ids:
                    if segments_info[frame_idx][pan_id-1]['isthing']:
                        pan_single_mask = pan_mask == pan_id
                        dist = center_distance(box,pan_single_mask)
                        distances.append(dist)
                
            #  print(min(distances))
                if (len(distances) != 0):
                    if min(distances) < 40: # distance small enough
                        box_nr = pan_ids[distances.index(min(distances))]
                        segments_info[frame_idx][box_nr-1].update( {'track_id' : track_id} )
                # print(box_nr)
                # print(track_id)

    path = info_path_list[v][:-4]+'_deepsort' #Zurück ändern sobald roundabout da ist
    print(path)
    np.save(path,segments_info,allow_pickle=True)

/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/panoptic-seg/street_data.npy
/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/panoptic-seg/street_info.npy
/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/deep_sort/detections/street.txt



/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/panoptic-seg/street_info_deepsort



## Visualize Deepsort Video
